In [11]:
import pandas as pd
import numpy as np
import time
import pulp  

processing_time=pd.read_excel("Jobshop.xlsx",
                              sheet_name="processing_time",
                              header = 0)
machine_order=pd.read_excel("Jobshop.xlsx",
                            sheet_name="machine_order",
                            header = 0)

pt = processing_time.values
mo = machine_order.values

model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)

dv = pulp.LpVariable.dicts("start_time",
                                     ((i, j) for i in range(6) for j in range(6)),
                                     lowBound=0,
                                     cat='Continuous')
b =  pulp.LpVariable.dicts("binary_var",
                                     ((i, j,o) for i in range(6) for j in range(6) for o in range(6) if j<o),
                                     lowBound=0,
                                     cat='Binary')

Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')


model += Cmax

for j in range(6):
    for i in range(5):
       I = mo[j,i]
       K = mo[j,i+1]
       J = j
       model += (dv[K,J] - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for j in range(6):
    for i in range(6):
       I = mo[j,i]
       J = j
       model += (Cmax - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for i in range(6):
    for j in range(6):
        for l in range(6):
            if j<l:
                I=i
                L = l
                J = j
                model += (dv[I,J] - dv[I,L]) >= (pt[L][list(mo[L]).index(I)] - 100*(1-b[i,J,L]))
                model += (dv[I,L] - dv[I,J]) >= (pt[J][list(mo[J]).index(I)] - 100*(b[i,J,L]))
       
start_time = time.time()
model.solve()
pulp.LpStatus[model.status]     

for var in dv:
    var_value = dv[var].varValue
    print( var[0],'-',var[1] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print('the elapsed time:%s'% (time.time() - start_time))

KeyError: (6, 0)

In [9]:
import pandas as pd
import numpy as np
import time
import pulp  

processing_time=pd.read_excel("Jobshop.xlsx",
                              sheet_name="processing_time",
                              header = 0)
machine_order=pd.read_excel("Jobshop.xlsx",
                            sheet_name="machine_order",
                            header = None)
processing_time

,Job,Machine,Machine.1,Machine.2,Machine.3,Machine.4,Machine.5
0,1,3,1,2,4,6,5
1,2,2,3,5,6,1,4
2,3,3,4,6,1,2,5
3,4,2,1,3,4,5,6
4,5,3,2,5,6,1,4
5,6,2,4,6,1,5,3


In [12]:
import pandas as pd
import numpy as np
import time
import pulp  

processing_time=pd.read_excel("Jobshop.xlsx",
                              sheet_name="processing_time",
                              header = 0)
machine_order=pd.read_excel("Jobshop.xlsx",
                            sheet_name="machine_order",
                            header = 0)

pt = processing_time.values
mo = machine_order.values

model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)

dv = pulp.LpVariable.dicts("start_time",
                                     ((i, j) for i in range(6) for j in range(6)),
                                     lowBound=0,
                                     cat='Continuous')
b =  pulp.LpVariable.dicts("binary_var",
                                     ((i, j,o) for i in range(6) for j in range(6) for o in range(6) if j<o),
                                     lowBound=0,
                                     cat='Binary')

Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')


model += Cmax

#加入限制式
#確保工作的Operation順序，必須前一站完成才能排下一站
for j in range(6):
    for i in range(5):
       I = mo[j,i]
       K = mo[j,i+1]
       J = j
       model += (dv[K,J] - dv[I,J]) >= pt[I,J]
#定義makespan       
for j in range(6):
    for i in range(6):
       I = mo[j,i]
       J = j
       model += (Cmax - dv[I,J]) >= pt[I,J]
#確保一台機台一次只能有一個工作進行       
for i in range(6):
    for j in range(6):
        for l in range(6):
            if j!=l:
                I=i
                L = l
                J = j
                model += (dv[I,J] - dv[I,L]) >= (pt[I,L] - 100*(1-b[i,J,L]))
                model += (dv[I,L] - dv[I,J]) >= (pt[I,J] - 100*(b[i,J,L]))
       
#start_time = time.time()
model.solve()
pulp.LpStatus[model.status]     

for var in dv:
    var_value = dv[var].varValue
    print( var[0],'-',var[1] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
#print('the elapsed time:%s'% (time.time() - start_time))

KeyError: (6, 0)